# "[RNN] LSTM"
> "문장 예측하기"
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [DL, Pytorch, LSTM]
- author: 도형준

In [ ]:
# https://www.kaggle.com/{본인ID}/account
# 본인 프로필 누르고 > Account
# > Create New API Token > kaggle.json
from google.colab import files

!pip install -q kaggle #kaggle 설치
# -- kaggle api를 사용하기 위한 인증파일을 설정
files.upload() #kaggle API file upload
!mkdir ~/.kaggle # kaggle 디렉토리 생성 / mkdir : make directory - 폴더 생성 
!cp kaggle.json ~/.kaggle/ #kaggle.json 파일 kaggle 폴더에 복사 / cp a b
!chmod 600 ~/.kaggle/kaggle.json # 권한 변경 r w x
# ---------
# !kaggle datasets list #kaggle 데이터셋 리스트 체크
# Copy API commend > 예) kaggle datasets download -d aashita/nyt-comments
!kaggle datasets download -d aashita/nyt-comments
!ls # 다운받은 파일 리스트 확인 -> nyt-comments.zip

Saving kaggle.json to kaggle.json
 98% 468M/480M [00:02<00:00, 210MB/s]
100% 480M/480M [00:02<00:00, 208MB/s]
kaggle.json  nyt-comments.zip  sample_data


In [ ]:
!unzip nyt-comments # unzip (확장없는파일명)

Archive:  nyt-comments.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


In [ ]:
# 데이터 살펴보기
import pandas as pd

df = pd.read_csv('ArticlesApril2017.csv')
df.columns # headline => 사람이 작성한 (자연어) 기사 데이터
df.headline

0      Finding an Expansive View  of a Forgotten Peop...
1                      And Now,  the Dreaded Trump Curse
2                  Venezuela’s Descent Into Dictatorship
3                  Stain Permeates Basketball Blue Blood
4                              Taking Things for Granted
                             ...                        
881                  Reporting on Gays Who ‘Don’t Exist’
882    The Fights That Could Lead to a Government Shu...
883    ‘The Leftovers’ Season 3, Episode 2: Swedish P...
884                          Thinking Out Loud, But Why?
885                    Some Sugar. Could Use More Spice.
Name: headline, Length: 886, dtype: object

In [ ]:
# 데이터 전처리
# 1. 특수문자 제거 (string...)
# 2. BOW (Bag of words) - 모든 단어를 겹치지 않도록 고유한 번호로 나타낸 집합
# - 딥러닝을 위해 만들어진 고유 번호(특정한 단어)를 담은 사전
# 예)
# 철수가 '사과'를 먹었다.
# -> 철수가 사과를 먹었다
# -> <철수>가(0) <사과>를(1) <먹>었다(2)

import numpy as np
import glob

from torch.utils.data.dataset import Dataset
import string

In [ ]:
def clean_text(txt): # 특수문자 제거용 함수 정의
    # 모든 단어를 소문자로 바꾸고 특수 문자를 제거
    new_text = []
    for v in txt:
        if v not in string.punctuation:
            new_text.append(v)
    return "".join(new_text).lower()

clean_text('Hello, hello! hello?')

'hello hello hello'

In [ ]:
glob.glob("*.csv")

['ArticlesMay2017.csv',
 'CommentsApril2017.csv',
 'CommentsFeb2018.csv',
 'ArticlesMarch2018.csv',
 'CommentsMay2017.csv',
 'CommentsJan2018.csv',
 'ArticlesFeb2017.csv',
 'ArticlesJan2018.csv',
 'CommentsFeb2017.csv',
 'ArticlesMarch2017.csv',
 'ArticlesApril2017.csv',
 'CommentsMarch2017.csv',
 'CommentsJan2017.csv',
 'ArticlesFeb2018.csv',
 'CommentsMarch2018.csv',
 'ArticlesApril2018.csv',
 'CommentsApril2018.csv',
 'ArticlesJan2017.csv']

In [ ]:
class TextGeneration(Dataset):
    def clean_text(self, txt): # 특수문자 제거용 함수 정의
        # 모든 단어를 소문자로 바꾸고 특수 문자를 제거
        # new_text = [] # 새로운 문자를 받을 리스트
        # for v in txt: # txt 집합을 한 단어씩 순회
        #     if v not in string.punctuation: # 특수문자집합(punctuation)에 속하지 않으면
        #         new_text.append(v) # 새로운 텍스트에 포함시킴
        # return "".join(new_text).lower()
        # 모든 순회가 끝나고 "(빈칸)"을 기준으로 합쳐준다음,
        # lower()로 모두 소문자로 만듦
        return "".join(
            [v for v in txt if v not in string.punctuation]
            ).lower()
    # 생성자 -> 클래스를 통해서 객체 -> 내부에 들어있는 변수들 지정해주는 과정
    def __init__(self):
        all_headlines = []
        # 모든 헤드라인의 텍스트 불러오기
        for filename in glob.glob("*.csv"):
            if 'Articles' in filename:
                article_df = pd.read_csv(filename)
                # 안에 있는 원소들을 바로 리스트 연결
                all_headlines.extend(list(article_df.headline.values))
                break
        # healine 중 unknown 값을 제거
        all_headlines = [h for h in all_headlines if h != "Unknown"]
        # 구두점 제거 등 전처리
        self.corpus = [self.clean_text(x) for x in all_headlines]
        self.BOW = {}
        # 모든 문장의 단어 추출 후 고유 번호 지정
        for line in self.corpus:
            for word in line.split(): # " " 기준으로 라인들을 분리 -> 단어
                if word not in self.BOW.keys(): # 딕셔너리(사전형) -> 키
                    self.BOW[word] = len(self.BOW.keys())
                    # 1 : 사과 -> 사과 : 0 (key:1)
                    # 2:  배   -> 배   : 1 (key:2)
                    # 3 : 사과 -> pass
                    # 4 ...
        # 모델의 입력으로 사용할 데이터(self.data)
        self.data = self.generate_sequence(self.corpus)

    # 텍스트 시계열 -> BOW
    # (몇 개의 단어) -> 그 다음 단어
    def generate_sequence(self, txt):
        seq = []
        for line in txt: # txt -> 전처리된 헤드라인(제목)
            # 1단계
            line = line.split() # space 기준으로 분할 -> 단어
            # BOW(사전) -> key -> key : 고유번호 apple 1 is 2 => 1, 2
            line_bow = [self.BOW[word] for word in line]
            # 단어의 리스트를 고유 번호의 리스트
            # (배, 감, 사과), 바나나
            # 배, (감, 사과, 바나나)
            # 단어 2개를 입력 -> 그 다음 단어가 정답(예측 대상)
            data = [([line_bow[i], line_bow[i+1]], line_bow[i+2])
                for i in range(len(line_bow)-2)]
            # data = [([line_bow[i], line_bow[i+1], line_bow[i+2]], line_bow[i+3])
            #     for i in range(len(line_bow)-3)] # 3개 -> 1개
            seq.extend(data) # (1번단어, 2번단어 -> 3번단어)
        return seq
    # 데이터 개수를 반환하는 함수
    def __len__(self):
        return len(self.data)
    # 데이터를 불러오는 함수
    def __getitem__(self, i):
        data = np.array(self.data[i][0]) # (1번, 2번 단어)
        label = np.array(self.data[i][1]) # (3번 단어)
        return data, label

In [ ]:
import torch.nn as nn
import torch

# 모델 정의
class LSTM(nn.Module):
    def __init__(self, num_embeddings):
        # [임베딩층]
        # 자연어 처리 -> 단어 개수 -> 엄청 많다
        # 모델에 들어가는 대부분의 입력값이 0이 됌
        # => 희소 표현 (학습 제대로 X) => 가중치를 곱해도 0
        # 희소표현 => 밀집표현 (밀집표현 : 0 거의 X)
        super(LSTM, self).__init__()

        # 밀집표현을 위한 임베딩 층
        self.embed = nn.Embedding(
            num_embeddings=num_embeddings, embedding_dim=16)
        # LSTM 5개 층을 쌓음
        self.lstm = nn.LSTM(
            input_size=16,
            hidden_size=64,
            num_layers=5,
            batch_first=True)
        # batch, sequence, feature
        #   -        2       64
        # 분류 - FC, MLP
        self.fc1 = nn.Linear(128, num_embeddings)
        self.fc2 = nn.Linear(num_embeddings, num_embeddings)

        # 활성화 함수
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.embed(x) # 희소 -> 밀집

        # LSTM 모델의 예측값
        x, _ = self.lstm(x) # 전체 출력 상태 + 셀 상태값
        x = torch.reshape(x, (x.shape[0], -1)) # batch, 2 * 64 / batch, 128
        x = self.fc1(x) # 128 -> 예측하려고 하는 문장 길이 
        x = self.relu(x)
        x = self.fc2(x) # 문장 길이 => 문장 길이

        return x

In [ ]:
# 학습 루프
from tqdm.notebook import tqdm

from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'

dataset = TextGeneration() # 데이터셋 정의
model = LSTM(num_embeddings=len(dataset.BOW)).to(device)
loader = DataLoader(dataset, batch_size=64)
optim = Adam(model.parameters(), lr=0.001)

for epoch in range(200):
    iterator = tqdm(loader)
    # data : 입력값 / label : 정답값
    for data, label in iterator:
        optim.zero_grad() # 기울기 초기화
        # 모델의 예측값
        pred = model(torch.tensor(data, dtype=torch.long).to(device))
        # 정답 레이블 -> long 텐서
        loss = nn.CrossEntropyLoss()(
            pred, torch.tensor(label, dtype=torch.long).to(device)
        )
        # 오차 역전파
        loss.backward()
        optim.step()
        
        iterator.set_description(f"epoch{epoch} loss:{loss.item()}")

torch.save(model.state_dict(), 'lstm.pth')

  0%|          | 0/72 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

...

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

In [ ]:
# 문장 만들기 (2) -> (1) ...
def generate(model, BOW, string="finding an ", strlen=10):
    device = "cuda" if torch.cuda.is_available() else 'cpu'
    print(f"input word: {string}")

    with torch.no_grad():
        for p in range(strlen):
            # 입력한 문장을 텐서화
            words = torch.tensor([BOW[w] for w in string.split()], dtype=torch.long).to(device)
            # 모델의 입력으로 사용하기 위한 배치 차원을 추가
            input_tensor = torch.unsqueeze(words[-2:], dim=0)
            output = model(input_tensor) # 모델을 통해 예측
            output_word = (torch.argmax(output).cpu().numpy())
            string += list(BOW.keys())[output_word]
            string += " "
    
    print(f"predicted sentence: {string}")

In [ ]:
model.load_state_dict(torch.load("lstm.pth", map_location=device))
pred = generate(model, dataset.BOW)

input word: finding an 
predicted sentence: finding an in future master bright blame knees go demurred no arthritis 


In [ ]:
pred = generate(model, dataset.BOW, string="bull market ")

input word: bull market 
predicted sentence: bull market a shutdown now calls for one later the president’s affair 
